In [10]:
import baml_client.types as models
import rdflib
from scripts import parse_module_catalog, parse_overview
from requests.auth import HTTPBasicAuth

In [34]:
uri = "http://ramp.uni-mannheim.de"
graph_endpoint = "https://graph.h4hn.de/bigdata/namespace/ramp/sparql"
username = "ramp"
password = "d2JKXsiF8vZJbu8NyWMN"

In [2]:
overview = parse_overview("./output/20241112_000328/catalog_overview.json")

In [7]:
def generate_sparql_insert_from_study_program_overview(study_program_overview: models.StudyProgrammOverview) -> str:
    sparql_statements = []
    
    # Loop through each study area in the study program
    for study_area in study_program_overview.studyArea:
        # Loop through each module in the study area
        for module in study_area.modules:
            # Basic SPARQL insert for each module
            sparql = (
                f"INSERT DATA {{\n"
                f"    <{uri}/module/{module.id}> a :Module ;\n"
                f"    :name \"{module.name}\" ;\n"
                f"    :id \"{module.id}\" .\n"
                f"}}\n"
            )
            sparql_statements.append(sparql)
    
    # Combine all statements
    return "\n".join(sparql_statements)

sparql = generate_sparql_insert_from_study_program_overview(overview)

print(sparql)


INSERT DATA {
    <http://ramp.uni-mannheim.de/module/CS_560> a :Module ;
    :name "large_scale_data_management" ;
    :id "CS_560" .
}

INSERT DATA {
    <http://ramp.uni-mannheim.de/module/IE_500> a :Module ;
    :name "data_mining" ;
    :id "IE_500" .
}

INSERT DATA {
    <http://ramp.uni-mannheim.de/module/IE_675b> a :Module ;
    :name "machine_learning" ;
    :id "IE_675b" .
}

INSERT DATA {
    <http://ramp.uni-mannheim.de/module/CS_652> a :Module ;
    :name "data_security_and_privacy" ;
    :id "CS_652" .
}

INSERT DATA {
    <http://ramp.uni-mannheim.de/module/AC_651> a :Module ;
    :name "additional_course_data_management" ;
    :id "AC_651" .
}

INSERT DATA {
    <http://ramp.uni-mannheim.de/module/DS_200> a :Module ;
    :name "sampling_and_data" ;
    :id "DS_200" .
}

INSERT DATA {
    <http://ramp.uni-mannheim.de/module/CS_500> a :Module ;
    :name "advanced_software_engineering" ;
    :id "CS_500" .
}

INSERT DATA {
    <http://ramp.uni-mannheim.de/module/CS_550> a

In [35]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF
import requests

RAMP = Namespace("http://ramp.uni-mannheim.de/")

#upload to blazegraph
# Initialize an RDFLib Graph
graph = Graph()
graph.bind("ramp", RAMP)

# Iterate through each study area and module to create triples
for study_area in overview.studyArea:
    for module in study_area.modules:
        # Create the URI for the module
        module_uri = URIRef(f"http://example.org/module/{module.id}")

        # Add RDF type triple
        graph.add((module_uri, RDF.type, RAMP.Module))
        
        # Add literal properties for name and id
        graph.add((module_uri, RAMP.name, Literal(module.name)))
        graph.add((module_uri, RAMP.id, Literal(module.id)))

# Serialize and print the graph in Turtle format (optional for debugging)
print(graph.serialize(format="turtle"))

rdf_data = graph.serialize(format="nt")

# Send the update request to the SPARQL endpoint
# Prepare SPARQL INSERT statement
sparql_update = f"""
INSERT DATA {{
    {rdf_data}
}}
"""

# Send the request with Basic Authentication
headers = {
    "Content-Type": "application/sparql-update"
}

response = requests.post(
    url=graph_endpoint,
    data=sparql_update,
    headers=headers,
    auth=HTTPBasicAuth(username, password)
)

if response.status_code == 200:
        print("Data successfully inserted.")
else:
    print("Failed to insert data:", response.status_code, response.text)


@prefix ramp: <http://ramp.uni-mannheim.de/> .

<http://example.org/module/AC_651> a ramp:Module ;
    ramp:id "AC_651" ;
    ramp:name "additional_course_data_management" .

<http://example.org/module/AC_652> a ramp:Module ;
    ramp:id "AC_652" ;
    ramp:name "additional_course_data_analytics_methods" .

<http://example.org/module/AC_653> a ramp:Module ;
    ramp:id "AC_653" ;
    ramp:name "additional_course_projects_and_seminars" .

<http://example.org/module/AC_654> a ramp:Module ;
    ramp:id "AC_654" ;
    ramp:name "additional_course_responsible_data_science" .

<http://example.org/module/AC_655> a ramp:Module ;
    ramp:id "AC_655" ;
    ramp:name "additional_course_data_science_applications" .

<http://example.org/module/CS_500> a ramp:Module ;
    ramp:id "CS_500" ;
    ramp:name "advanced_software_engineering" .

<http://example.org/module/CS_550> a ramp:Module ;
    ramp:id "CS_550" ;
    ramp:name "algorithmics" .

<http://example.org/module/CS_560> a ramp:Module ;
    r